# Investigating accident rates

Now that you have the number of accidents for some road segments, you can investigate and find accident hotspots.

First, some boilerplate imports.

In [8]:
# Import the required libraries and open the connection to Mongo

import collections
import datetime
import matplotlib as mpl

import pandas as pd
import scipy.stats

import folium

import pymongo

In [9]:
# Open a connection to the Mongo server, open the accidents database and name the collections of accidents and labels
client = pymongo.MongoClient('mongodb://localhost:27351/')

db = client.accidents
accidents = db.accidents
labels = db.labels
roads = db.roads

In [10]:
# Load the expanded names of keys and human-readable codes into memory
expanded_name = collections.defaultdict(str)
for e in labels.find({'expanded': {"$exists": True}}):
    expanded_name[e['label']] = e['expanded']
    
label_of = collections.defaultdict(str)
for l in labels.find({'codes': {"$exists": True}}):
    for c in l['codes']:
        try:
            label_of[l['label'], int(c)] = l['codes'][c]
        except ValueError: 
            label_of[l['label'], c] = l['codes'][c]

In [11]:
def add_accidents_markers(the_map, query, number_of_sides=5, fill_color='#769d96', limit=0,
                     radius=5, rotation=54):
    for a in accidents.find(query, 
                            ['loc.coordinates'],
                            limit=limit):
        folium.RegularPolygonMarker(location=[a['loc']['coordinates'][1], a['loc']['coordinates'][0]], 
                     number_of_sides=number_of_sides, radius=radius, rotation=rotation,
                                   fill_color=fill_color).add_to(the_map)  

In [12]:
def add_roads_markers(the_map, query, number_of_sides=5, fill_color='#769d96', limit=0,
                     radius=5, rotation=54):
    for r in roads.find(query, 
                        ['loc.coordinates'],
                       limit=limit):
        folium.RegularPolygonMarker(location=[r['loc']['coordinates'][1], r['loc']['coordinates'][0]], 
                     number_of_sides=number_of_sides, radius=radius, rotation=rotation,
                                   fill_color=fill_color).add_to(the_map)   

## Exploring accident counts
Now that most of the road segments are annotated with the accidents that are nearby, we can investigate accident rates.

In [43]:
# Build a DataFrame, one row for each accident
nac_unrolled_df = pd.DataFrame(list(roads.find({}, ['nearby_accident_count'])))
roads.find({'nearby_accident_count' : {'$exists': True}}).count()
# Count the number of each severity
#nac_ss = nac_unrolled_df['nearby_accident_count'].value_counts()
#nac_ss

0

In [23]:
nac_ss.plot()

NameError: name 'nac_ss' is not defined

It's obvious that most road segments have only a few accidents allocated to them, while a few have many.

Let's take a look at where the most accident-prone road segments are.

In [22]:
m = folium.Map([55, -3], zoom_start=6)    
add_roads_markers(m, {'loc': {'$exists': True},
                      'nearby_accident_count': {'$gte': 40}})
m

Nothing particularly illuminating. Most of the accident-prone road sections are in central London, which isn't a great surprise.

In [26]:
milton_keynes = {'type': 'Polygon',
                 'coordinates': [[[-0.877025, 52.092317],
                                  [-0.651709, 52.092317],
                                  [-0.651709, 51.958628],
                                  [-0.877025, 51.958268],
                                  [-0.877025, 52.092317]
                                 ]]}

min_mk_lat = min(p[1] for p in milton_keynes['coordinates'][0])
max_mk_lat = max(p[1] for p in milton_keynes['coordinates'][0])
min_mk_lon = min(p[0] for p in milton_keynes['coordinates'][0])
max_mk_lon = max(p[0] for p in milton_keynes['coordinates'][0])

mk_centre = [min_mk_lat + (max_mk_lat - min_mk_lat) / 2, min_mk_lon + (max_mk_lon - min_mk_lon) / 2]

mk_region_query = {'loc': {'$geoWithin': {'$geometry': milton_keynes}}}

In [27]:
import math 

m = folium.Map(mk_centre, zoom_start=9)    
query = {'loc':
                      {'$nearSphere':
                       {'$geometry': 
                        {'type': 'Point', 'coordinates': list(reversed(mk_centre))},
                        '$maxDistance': 50000}}}

for r in roads.find(query):
    folium.RegularPolygonMarker(location=[r['loc']['coordinates'][1], r['loc']['coordinates'][0]], 
                     number_of_sides=9, fill_color='#ff0000',
                    radius=(math.log10(r['nearby_accident_count']+1)*6)).add_to(m)

m

KeyError: 'nearby_accident_count'

Nothing immediately obvious leaps out about accident hotspots. There seem to be large numbers of accidents near towns, but that is likely because there are lots of road segments and drivers near towns. 

Raw 'number of accidents' per road section isn't that interesting, as the road sections are different lengths. We could look at the number of accidents per kilometre, or the number of accidents per vehicle-kilometre. 

In [38]:
pipeline = []
pipeline = [{'$project': {'_id': '$CP',
                          'accidents_per_km': {'$divide': ['$nearby_accident_count', '$LenNet']}}}]

a_per_km_df = pd.DataFrame(list(roads.aggregate(pipeline)))
a_per_km_df.set_index('_id', inplace=True)
a_per_km_df.index.name = 'CP'
a_per_km_df

,accidents_per_km
CP,
506,NaN
508,NaN
509,NaN
510,NaN
512,NaN
514,NaN
515,NaN
82013,NaN
516,NaN


In [ ]:
a_per_km_df['accidents_per_km'].hist()
a_per_km_df['accidents_per_km'].describe()

There's a road with 310 accidents per kilometre? Where?

In [ ]:
a_per_km_df.loc[a_per_km_df['accidents_per_km'].idxmax()]

In [ ]:
roads.find_one({'CP': int(a_per_km_df['accidents_per_km'].idxmax())})

### Activity 1
Find the number of accidents per kilometre, and the number of accidents per vehicle-kilometre, for each road category.

(Use the `FdAll_MV` data for the number of vehicles using a road section.)

The solution is in the [`15.5solutions`](15.5solutions.ipynb) Notebook.

In [ ]:
# Insert your solution here.

### Activity 2
Plot a map of the road sections within 50km of Milton Keynes with accidents, with the size of marker dependent on the accidents per vehicle-km on that road section. If a road section doesn't have a meaningful accident per vehicle-km, it should be discarded from the display.

Note that the accident rate per vehicle-kilometre spans several orders of magnitude, so you should use a marker size based on the logarithm of the rate. 

The solution is in the [`15.5solutions`](15.5solutions.ipynb) Notebook.

In [ ]:
# Insert your solution here.

### Activity 3
What's the most dangerous road segment in Milton Keynes?

The solution is in the [`15.5solutions`](15.5solutions.ipynb) Notebook.

In [ ]:
# Insert your solution here.

### Activity 4
What are the proportions of casualty class (e.g. `accident.Casualty_Class`) for accidents that occur on different road classes? Are there significant differences between the proportions of casualty classes?

The solution is in the [`15.5solutions`](15.5solutions.ipynb) Notebook.

In [ ]:
mpl.rcParams['figure.figsize'] = (8, 8)

In [ ]:
# Insert your solution here.

## Summary
The combination of traffic volume data and accident data allows us to get a richer and more nuanced view of road safety. Rather than just looking for areas with many accidents, we can look at where the accident rate is highest compared to the traffic volumes. 

This is a typical part of data investigations and is one you'll come back to in this module, particularly in the EMA.

## What next?
If you are working through this Notebook as part of an inline exercise, return to the module materials now.

If you are working through this set of Notebooks as a whole, you've completed the Part 15 Notebooks. It's time to move on to Part 16.